In [ ]:
!pip install -q langchain langchain-community langchain-groq duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from langchain_groq import ChatGroq
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.runnables.history import RunnableWithMessageHistory
import os
import getpass


In [ ]:
print("🔑 Please enter your Groq API key:")
print("(You can get one free at: https://console.groq.com/)")
groq_api_key = getpass.getpass("Groq API Key: ")
os.environ["GROQ_API_KEY"] = groq_api_key
print("✅ API key set successfully!")

🔑 Please enter your Groq API key:
(You can get one free at: https://console.groq.com/)
Groq API Key: ··········
✅ API key set successfully!


In [ ]:
@tool
def physics_search(query: str) -> str:
    """Search for physics info, formulas, experiments, concepts."""
    search = DuckDuckGoSearchRun()
    return search.run(query)

tools = [physics_search]

In [1]:
class PhysicsTeacherAgent:
    def __init__(self):
        # 🧠 Groq LLM
        self.llm = ChatGroq(
            model="llama-3.1-8b-instant",
            temperature=0.7,
            max_tokens=1024,
        )

        # 🎓 Persona Setup
        self.system_message = SystemMessage(content="""
        You are an expert physics teacher.
        - Explain concepts clearly using simple words.
        - Always include formulas when needed.
        - Search online when necessary using the `physics_search` tool.
        - Provide examples when relevant.
        - You must teach like a human teacher, step by step.
        """)

        #  Prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            self.system_message,
            ("human", "{question}")
        ])

        # 🔗 Chain without deprecated agent API
        self.chain = self.prompt | self.llm


        self.store = {}

    # Memory Getter
    def _get_history(self, session_id="student1"):
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]

    # Runnable With Memory
    @property
    def runnable(self):
        return RunnableWithMessageHistory(
            self.chain,
            self._get_history,
            input_messages_key="question",
        )

    # 💬 Ask a Question
    def ask_question(self, question: str, session_id="student1"):
        query = f"Physics question: {question}. Search online if needed and explain clearly."
        response = self.runnable.invoke(
            {"question": query},
            config={"configurable": {"session_id": session_id}}
        )
        return response.content

    # 🧾 Get chat history
    def get_conversation_history(self, session_id="student1"):
        return self._get_history(session_id).messages


# 🚀 Initialize Agent
print("🔬 Initializing Physics Teacher Agent...")
physics_teacher = PhysicsTeacherAgent()
print("✅ Physics Teacher Agent Ready!")

🔬 Initializing Physics Teacher Agent...


NameError: name 'ChatGroq' is not defined

In [ ]:
print(physics_teacher.ask_question("What is Newton's second law?"))


Newton's second law is a fundamental concept in physics that describes the relationship between the motion of an object and the forces acting upon it.

**What is Newton's Second Law?**

Newton's second law states that the acceleration of an object is directly proportional to the net force acting upon it and inversely proportional to its mass.

**Mathematical Formula:**

The formula for Newton's second law is:

F = ma

Where:

- F (Force) is the net force acting on the object
- m (Mass) is the mass of the object
- a (Acceleration) is the acceleration of the object

**Breaking it Down:**

Let's break down the formula:

- **Force (F)**: The net force acting on an object is the sum of all the forces acting on it. For example, if you're pushing a box, the force you're applying to the box is the net force.
- **Mass (m)**: The mass of an object is a measure of its resistance to changes in its motion. The more massive an object is, the more force is required to produce a given acceleration.
- 

In [ ]:
print(physics_teacher.ask_question("Give one real-world example"))


I'd be happy to provide a real-world example of Newton's second law.

**Example: A Car Accelerating**

Imagine you're driving a car with a mass of 1500 kg. You press the gas pedal, and the car accelerates from 0 to 60 km/h in 10 seconds. What is the net force acting on the car, and what is its acceleration?

Let's use the formula F = ma to solve this problem.

First, we need to convert the mass of the car from kilograms to grams (since we're using the International System of Units, or SI units). However, in this case, we can use kilograms directly.

We also need to convert the acceleration from km/h to m/s². To do this, we'll use the following conversion factors:

1 km/h = 0.27778 m/s
1 hour = 3600 seconds

So, we can convert 60 km/h to m/s as follows:

60 km/h = 60 x 0.27778 m/s = 16.667 m/s

Now, we can calculate the acceleration of the car using the formula:

a = Δv / Δt
= (16.667 m/s - 0 m/s) / 10 s
= 1.667 m/s²

Next, we need to calculate the net force acting on the car. According